# Ensembles: Random Forest

Random Forest es uno de los modelos más usados en la actualidad y suele brindar excelentes resultados.

### Fundamento de Random Forest:

Con Random Forest la idea es la de crear un ensemble con muchos árboles distintos entre sí.  

Las diferencias entre los árboles se inducen **aleatoriamente** con dos mecanismos:  

- Por un lado se seleccionan aleatoriamente **observaciones o instancias**
- Por otro lado se seleccionan aleatoriamente **variables o features**

Es decir que cada uno de los múltiples árboles que formarán el ensemble (forest) se generará basado en distintas observaciones y distintas variables o features.



- Generalmente las observaciones con que se entrenará cada árbol se seleccionan con un mecanismo de **bootstrap**, es decir que cuando se selecciona una observación al azar dentro del Train Set original para incluirla en el Train Set de un árbol en particular, no se la desecha, sino que se la vuelve a incluir entre las posibles candidatas a ser elegida, de esta manera al generar las observaciones para uno de los árboles puede ocurrir que una observación del Train Set original participe más de una vez en el Train Set de un árbol en particular. Es por esto que muchas veces el tamaño de las observaciones para entrenar cada árbol suele elegirse con la misma cantidad de elementos que todo el train set original, sin que esto indique que todos los árboles se entrenen con las mismas observaciones.  


- Al seleccionar sólo un subconjunto de todas las features para cada uno de los árboles individuales obtenemos una ventaja adicional: Como los árboles tienen tendencia al overfitting, al crear estos árboles simplificados con menos variables, disminuyen los grados de libertad del modelo y por lo tanto tiende a disminuir la varianza; y aunque también tiende a aumentar el Bias, generalmente es muy conveniente.


De la misma manera que en los ensembles ya comentados, una vez obtenidos los pronósticos con todos los árboles se efectúa un mecanismo de votación si era un problema de Clasificación o se promedian los valores pronosticados en el caso de un problema de Regresión. 



### Random Forest en Scikit-Learn:

Encontrará la documentación para el clasificador aquí: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html?highlight=random%20forest#sklearn.ensemble.RandomForestClassifier

La sintaxis es la siguiente:

~~~
from sklearn.ensemble import RandomForestClassifier

sklearn.ensemble.RandomForestClassifier(n_estimators=100, *, criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)
~~~

Los principales son, como podrá observar la mayoría son los mismos que para los árboles de decisión.

- **n_estimators=100** Es la cantidad de árboles que formarán el ensemble (forest)  
- **max_features='auto'**  Si se pasa un número entero, utilizará dicha cantidad, si se pasa un float, entonces se lo considerará como la fracción del total de features. En auto tomará la raíz cuadrada de todas las features. Si se pasa None, entonces utilizará todas las variables disponibles. 
- **bootstrap=True**  Si se pasa True, usa bootstrap para generar las observaciones, sino utiliza todas las observaciones.
- **max_samples=None** cuando bootstrap = True indica la cantidad de observaciones para cada árbol. Si se pasa None, entonces utiliza la misma cantidad de observaciones que las originales (aunque no necesariamente son las mismas!), si se pasa un entero, toma esa cantidad de observaciones, si se pasa un float, lo toma como fracción del total de observaciones.







### Ejemplo

Repitamos el problema de la clasificación de los vinos blancos:

#### Los datos:

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
df=pd.read_csv('data/winequality-white.csv', sep=';')
X=df.drop(axis=1,columns='quality')
y=df['quality']


Por las dudas vamos a desordenar un poco lo datos con Shuffle, porque me parece que siguen cierto patrón. 

In [ ]:
from sklearn.utils import shuffle
X,y=shuffle(X,y, random_state=123)

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=123, stratify=y)

### Creamos el modelo de Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_estimators=100,n_jobs=-1,random_state=123)

### Entrenamos con .fit

In [ ]:
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=123, verbose=0,
                       warm_start=False)

### Evaluemos Accuracy con Train - Test (Hold Out)

In [ ]:
rf.score(X_test,y_test)

0.6918367346938775

### Evaluemos con Cross Validation

In [ ]:
from sklearn.model_selection import cross_val_score 


In [ ]:
AC_scores = cross_val_score(rf, X, y, cv=5)
AC_media=AC_scores.mean()
AC_desvio=AC_scores.std()
# veamos los resultados
print("AC en cada fold: ",AC_scores)
print(AC_media,' x/- ',AC_desvio)

AC en cada fold:  [0.69387755 0.68673469 0.66734694 0.68028601 0.67722165]
0.6810933689103833  x/-  0.008942775925442277


### Con GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV

rf1=RandomForestClassifier(n_jobs=-1,random_state=123)
grid_rf1= GridSearchCV(estimator=rf1, param_grid={'n_estimators':[100,150,200,250,300,400,500]}, cv=5,n_jobs=-1, return_train_score=True)


In [ ]:
grid_rf1.fit(X,y)

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=-1,
                                              oob_score=False, random_state=123,
                                   

In [ ]:
AC=grid_rf1.best_score_
AC

0.6888522232181943

In [ ]:
grid_rf1.best_params_

{'n_estimators': 300}

### Conclusión

> Observemos que con Random Forest con 300 árboles hemos obtenido un valor sustancialmente mejor de Accuracy que con los restantes modelos individuales y con el ensemble que corrimos anteriormente. 

### Ejercicio:

Probar con GridSearchCV variando otros hiperparámetros de Random Forest, como por ejemplo max_features y max_samples, usar por lo menos dos o 3 valores para cada uno. 